In [49]:
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

Fill mising value

In [50]:
train_data = pd.read_csv('Train_samsung.csv')
numeric_columns = train_data.select_dtypes(include='number').columns
categorical_columns = train_data.select_dtypes(exclude='number').columns

imputer_numeric = SimpleImputer(strategy='mean')
imputer_categorical = SimpleImputer(strategy='most_frequent')

train_data[numeric_columns] = imputer_numeric.fit_transform(train_data[numeric_columns])
train_data[categorical_columns] = imputer_categorical.fit_transform(train_data[categorical_columns])

train_data[numeric_columns] = train_data[numeric_columns].round(1)
display(train_data)

header = ','.join(train_data.columns)
train_data.to_csv('Filled_train.csv', index=False, header=header)



,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,X11,Class
0,Male,Yes,0,Graduate,No,6000.0,2250.0,265.0,360.0,0.8,Semiurban,N
1,Male,Yes,0,Graduate,No,2958.0,2900.0,131.0,360.0,1.0,Semiurban,Y
2,Male,Yes,2,Graduate,No,6250.0,1695.0,210.0,360.0,1.0,Semiurban,Y
3,Male,Yes,0,Graduate,No,2083.0,3150.0,128.0,360.0,1.0,Semiurban,Y
4,Male,No,0,Graduate,No,4166.0,0.0,98.0,360.0,0.0,Semiurban,N
...,...,...,...,...,...,...,...,...,...,...,...,...
486,Male,Yes,2,Not Graduate,Yes,1875.0,1875.0,97.0,360.0,1.0,Semiurban,Y
487,Male,Yes,2,Graduate,No,11417.0,1126.0,225.0,360.0,1.0,Urban,Y
488,Female,No,0,Graduate,No,3237.0,0.0,30.0,360.0,1.0,Urban,Y
489,Female,Yes,0,Graduate,No,10047.0,0.0,148.7,240.0,1.0,Semiurban,Y


Separating features and labels

In [51]:
processed_data = pd.read_csv('Filled_train.csv')

X_train = processed_data.drop('Class', axis=1)
y_train = processed_data['Class']

X_train,y_train

(         X1   X2 X3            X4   X5       X6      X7     X8     X9  X10  \
 0      Male  Yes  0      Graduate   No   6000.0  2250.0  265.0  360.0  0.8   
 1      Male  Yes  0      Graduate   No   2958.0  2900.0  131.0  360.0  1.0   
 2      Male  Yes  2      Graduate   No   6250.0  1695.0  210.0  360.0  1.0   
 3      Male  Yes  0      Graduate   No   2083.0  3150.0  128.0  360.0  1.0   
 4      Male   No  0      Graduate   No   4166.0     0.0   98.0  360.0  0.0   
 ..      ...  ... ..           ...  ...      ...     ...    ...    ...  ...   
 486    Male  Yes  2  Not Graduate  Yes   1875.0  1875.0   97.0  360.0  1.0   
 487    Male  Yes  2      Graduate   No  11417.0  1126.0  225.0  360.0  1.0   
 488  Female   No  0      Graduate   No   3237.0     0.0   30.0  360.0  1.0   
 489  Female  Yes  0      Graduate   No  10047.0     0.0  148.7  240.0  1.0   
 490    Male  Yes  0      Graduate   No  13650.0     0.0  148.7  360.0  1.0   
 
            X11  
 0    Semiurban  
 1    Semiurba

Preprocess X_train

In [52]:
vectorizer = CountVectorizer()
X_train_vec = vectorizer.fit_transform(X_train)

Build discriminative model

In [53]:
X_test = pd.read_csv('Test_samsung_noclass.csv')

# Tạo vector đặc trưng từ dữ liệu test
X_test_vec = vectorizer.transform(X_test)


model = MultinomialNB()
model.fit(X_train_vec, y_train)

# Dự đoán nhãn cho dữ liệu test
y_test_pred = model.predict(X_test_vec)


for i, label in enumerate(y_test_pred):
    print(f"Sample {i+1}: {label}")

ValueError: Found input variables with inconsistent numbers of samples: [11, 491]